# Fix Direction Optimization on Sample Submission

Apply fix_direction rotation optimization to the sample submission to improve scores.

In [ ]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time

getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded")

In [ ]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree defined")

In [ ]:
def get_side_length(trees):
    if not trees:
        return Decimal('0')
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    return max(maxx - minx, maxy - miny)

def calculate_score(side_lengths):
    return sum(float(s)**2 / n for n, s in side_lengths.items())

def has_any_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Helper functions defined")

In [ ]:
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    if not trees:
        return Decimal('0'), 0.0
    
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)
    hull_points = points_np[ConvexHull(points_np).vertices]
    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    if initial_side - found_side > 1e-8:
        return Decimal(str(found_side)), found_angle_deg
    else:
        return Decimal(str(initial_side)), 0.0

def apply_rotation(trees, angle_deg):
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]

    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])

    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])

    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center / float(scale_factor)
    rotated = shifted.dot(rot_matrix.T) + rotation_center / float(scale_factor)

    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print("Rotation functions defined")

In [ ]:
# Load sample submission
def load_submission(filepath):
    df = pd.read_csv(filepath)
    df['x'] = df['x'].str.strip('s').astype(float)
    df['y'] = df['y'].str.strip('s').astype(float)
    df['deg'] = df['deg'].str.strip('s').astype(float)
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    configs = {}
    side_lengths = {}
    
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        trees = [ChristmasTree(center_x=str(row['x']), center_y=str(row['y']), angle=str(row['deg']))
                 for _, row in group_data.iterrows()]
        configs[n] = trees
        side_lengths[n] = get_side_length(trees)
    
    return configs, side_lengths

print("Loading sample submission...")
configs, side_lengths = load_submission('/home/data/sample_submission.csv')
initial_score = calculate_score(side_lengths)
print(f"Initial score: {initial_score:.6f}")

In [ ]:
# Apply fix_direction to all configurations
start_time = time.time()
improved_configs = {}
improved_side_lengths = {}

for n in range(1, 201):
    trees = configs[n]
    original_side = side_lengths[n]
    
    # Optimize rotation
    optimized_side, best_angle = optimize_rotation(trees)
    
    if optimized_side < original_side:
        improved_trees = apply_rotation(trees, best_angle)
        improved_configs[n] = improved_trees
        improved_side_lengths[n] = optimized_side
    else:
        improved_configs[n] = [t.clone() for t in trees]
        improved_side_lengths[n] = original_side
    
    if n % 50 == 0:
        elapsed = time.time() - start_time
        print(f"n={n}: original={float(original_side):.6f}, optimized={float(improved_side_lengths[n]):.6f}, elapsed={elapsed:.1f}s")

final_score = calculate_score(improved_side_lengths)
print(f"\nFinal score: {final_score:.6f}")
print(f"Improvement: {initial_score - final_score:.6f}")

In [ ]:
# Validate all configurations
invalid = []
for n, trees in improved_configs.items():
    if has_any_overlap(trees):
        invalid.append(n)

if invalid:
    print(f"WARNING: {len(invalid)} configurations have overlaps: {invalid[:10]}...")
else:
    print("All configurations are valid (no overlaps)")

In [ ]:
# Save submission
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]

tree_data = []
for n in range(1, 201):
    for tree in improved_configs[n]:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)

for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

submission.to_csv('/home/submission/submission.csv')
submission.to_csv('/home/code/experiments/003_fix_direction_sample/submission.csv')
print("Submission saved!")
print(f"\nFinal Score: {final_score:.6f}")